# TREC 2018 Precision Medicine

In [1]:
import json
from json2html import *
from IPython.display import HTML
import pandas

import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from trec_utils import utils, running, evaluation

In [2]:
config = utils.load_config()

## Load and split TOPICS and GOLD STANDARD from last year (30 topics)
Also, split them into training, test, and development set.

In [3]:
topics_all = utils.get_topics('./topics/topics2017.xml')
qrels_all = utils.get_qrels('./gold-standard/abstracts.2017.qrels')
#topics_all

In [4]:
topics_train, topics_test, topics_dev = utils.split_topics(topics_all)
qrels_train, qrels_test, qrels_dev = utils.split_qrels(qrels_all, topics_train, topics_test, topics_dev)

In [5]:
assert(set([topic for topic in topics_train['topic']])==set([topic for topic in qrels_train['topic']]))
assert(set([topic for topic in topics_test['topic']])==set([topic for topic in qrels_test['topic']]))
assert(set([topic for topic in topics_dev['topic']])==set([topic for topic in qrels_dev['topic']]))

## Experiment and do grid search

In [6]:
results = running.experiment(topics_all, qrels_all)

PARAMS GRID: {'query_template': ['variable.json'], 'disease_boost': [1, 1.5, 2], 'gene_tie_breaker': [0.2, 0.5, 1], 'gene_boost': [0.5, 1, 1.5, 2], 'disease_tie_breaker': [0.5, 1]}
RUN: variable.json-0.5-1-0.2-0.5 TOPICS: 30 {'query_template': 'variable.json', 'disease_boost': '1', 'gene_boost': '0.5', 'disease_tie_breaker': '0.5', 'gene_tie_breaker': '0.2', 'run_id': 'variable.json-0.5-1-0.2-0.5'}
(0.5414, 0.4967, 0.2764, '0.5', '1', '0.2', '0.5')
RUN: variable.json-0.5-1-0.2-1 TOPICS: 30 {'query_template': 'variable.json', 'disease_boost': '1', 'gene_boost': '1', 'disease_tie_breaker': '0.5', 'gene_tie_breaker': '0.2', 'run_id': 'variable.json-0.5-1-0.2-1'}
(0.5074, 0.4467, 0.2539, '0.5', '1', '0.2', '1')
RUN: variable.json-0.5-1-0.2-1.5 TOPICS: 30 {'query_template': 'variable.json', 'disease_boost': '1', 'gene_boost': '1.5', 'disease_tie_breaker': '0.5', 'gene_tie_breaker': '0.2', 'run_id': 'variable.json-0.5-1-0.2-1.5'}
(0.4783, 0.4033, 0.2364, '0.5', '1', '0.2', '1.5')
RUN: variab

In [7]:
results

,ndcg,P_10,Rprec,dis_tb,dis_b,gene_tb,gene_b
0,0.5414,0.4967,0.2764,0.5,1,0.2,0.5
12,0.5414,0.4967,0.2764,0.5,1.5,0.2,0.5
24,0.5414,0.4967,0.2764,0.5,2,0.2,0.5
38,0.5342,0.4600,0.2715,1,1,0.2,1.5
50,0.5342,0.4600,0.2715,1,1.5,0.2,1.5
62,0.5342,0.4600,0.2715,1,2,0.2,1.5
37,0.5316,0.4700,0.2678,1,1,0.2,1
49,0.5316,0.4700,0.2678,1,1.5,0.2,1
61,0.5316,0.4700,0.2678,1,2,0.2,1
39,0.5274,0.4767,0.2613,1,1,0.2,2


### Example runs

In [6]:
full_run_params = running.get_default_run_params()
full_run_params['query_template'] = 'baseline.json'
full_run_df, full_run_params = running.run(topics_all)
results, aggregated = evaluation.evaluate(qrels_all, full_run_df)
aggregated
#assert(aggregated['ndcg'] == 0.5338)
#{'P_10': 0.48, 'Rprec': 0.2847, 'ndcg': 0.5338, 'recall_1000': 0.6139}

RUN: DEFAULT_RUN TOPICS: 30 {'run_id': 'DEFAULT_RUN', 'gene_tie_breaker': 0.5, 'gene_boost': 1, 'query_template': 'baseline.json', 'disease_tie_breaker': 0.5, 'disease_boost': 1.5}


{'P_10': 0.48, 'Rprec': 0.2847, 'ndcg': 0.5338, 'recall_1000': 0.6139}

In [7]:
run_params = running.get_default_run_params()
run_params['query_template'] = 'baseline_sex_age.json'
full_run_df, run_params = running.run(topics_all)
results, aggregated = evaluation.evaluate(qrels_all, full_run_df)
aggregated
# Only MeSH, best fields: {'P_10': 0.5167, 'Rprec': 0.2785, 'ndcg': 0.5383, 'recall_1000': 0.6148}
# Only MeSH, cross fields: {'P_10': 0.5067, 'Rprec': 0.2811, 'ndcg': 0.5413, 'recall_1000': 0.6159}
# Only MeSH, cross, best: {'P_10': 0.52, 'Rprec': 0.2814, 'ndcg': 0.5378, 'recall_1000': 0.6151}
# Only MeSH, best, cross: {'P_10': 0.52, 'Rprec': 0.2787, 'ndcg': 0.5417, 'recall_1000': 0.6173} ***** best ndcg, best P_10

RUN: DEFAULT_RUN TOPICS: 30 {'run_id': 'DEFAULT_RUN', 'gene_tie_breaker': 0.5, 'gene_boost': 1, 'query_template': 'baseline_sex_age.json', 'disease_tie_breaker': 0.5, 'disease_boost': 1.5}


{'P_10': 0.52, 'Rprec': 0.2787, 'ndcg': 0.5417, 'recall_1000': 0.6173}

In [8]:
run_params = running.get_default_run_params()
run_params['query_template'] = 'baseline_sex_age_all_fields.json'
full_run_df, run_params = running.run(topics_all)
results, aggregated = evaluation.evaluate(qrels_all, full_run_df)
aggregated
# All fields, best fields: {'P_10': 0.51, 'Rprec': 0.2744, 'ndcg': 0.5396, 'recall_1000': 0.6143}
# All fields, cross fields: {'P_10': 0.5067, 'Rprec': 0.281, 'ndcg': 0.5412, 'recall_1000': 0.6157}
# All fields, best fields disease, gene, cross fields rest: {'P_10': 0.5167, 'Rprec': 0.276, 'ndcg': 0.5378, 'recall_1000': 0.6139}
# All fields, best fields disease, cross fields rest: {'P_10': 0.52, 'Rprec': 0.278, 'ndcg': 0.5416, 'recall_1000': 0.6174} ***** best ndcg, best P_10

RUN: DEFAULT_RUN TOPICS: 30 {'run_id': 'DEFAULT_RUN', 'gene_tie_breaker': 0.5, 'gene_boost': 1, 'query_template': 'baseline_sex_age_all_fields.json', 'disease_tie_breaker': 0.5, 'disease_boost': 1.5}


{'P_10': 0.52, 'Rprec': 0.278, 'ndcg': 0.5416, 'recall_1000': 0.6174}

In [6]:
#full_run_df

Test run with default_params

In [6]:
training_run_df, training_run_params = running.run(topics_train)

RUN: DEFAULT_RUN TOPICS: 12 {'run_id': 'DEFAULT_RUN', 'disease_tie_breaker': 0.5, 'gene_boost': 1, 'gene_tie_breaker': 0.5, 'query_template': 'baseline.json', 'disease_boost': 1.5}


In [7]:
training_results, training_aggregated = evaluation.evaluate(qrels_train, training_run_df)
training_score = pandas.Series(training_aggregated, name='training ' + '(' + str(len(topics_train)) + ' topics)')

In [8]:
test_run_df, test_run_params = running.run(topics_test, training_run_params)

RUN: DEFAULT_RUN TOPICS: 9 {'run_id': 'DEFAULT_RUN', 'disease_tie_breaker': 0.5, 'gene_boost': 1, 'gene_tie_breaker': 0.5, 'query_template': 'baseline.json', 'disease_boost': 1.5}


In [9]:
training_score

P_10           0.4333
Rprec          0.2810
ndcg           0.5383
recall_1000    0.6650
Name: training (12 topics), dtype: float64

In [10]:
test_results, test_aggregated = evaluation.evaluate(qrels_test, test_run_df)
test_score = pandas.Series(test_aggregated, name='test ' + '(' + str(len(topics_test)) + ' topics)')

In [11]:
pandas.DataFrame([training_score, test_score])

,P_10,Rprec,ndcg,recall_1000
training (12 topics),0.4333,0.2810,0.5383,0.6650
test (9 topics),0.3556,0.2194,0.4820,0.5423
